### MLE

In [1]:
import pytest

In [ ]:
def test_raise():
    with pytest.raises(ValueError):
        raise_exception()

In [2]:
import great_expectations as ge

In [ ]:
df = ge.read_table(data)

In [ ]:
df.exepct_column_values_to_be_unique("id")

In [ ]:
class TestFruit:
    def setup_method(self):
        self.fruit = Fruit("xx")
    
    def test_fruit(self):
        assert self.fruit.name == "xx"
    
    def teardown_method(self):
        del self.fruit

In [3]:
from typing import TypedDict

In [4]:
class NutritionInformation(TypedDict):
    value: int
    unit: str

In [5]:
class RecipeNutritionInformation(TypedDict):
    recipes_used: int
    calories: NutritionInformation
    carbs: NutritionInformation

In [ ]:
timestamp, value, key

### Engineering

In [6]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
#include <iostream>

In [ ]:
int main() {
    int i = 0;
    
    while (i <= 5) {
        std::cout << i;
        i++;
    }
    
    return 0;
}

In [ ]:
grid > thread block > thread

In [ ]:
int main() {
    int a = 69;
    int* b = &a;
    
    std::cout << *b;
    
    return 0;
}

In [ ]:
api server, scheduler, ectd, control manager

In [ ]:
class Monster() {
    public:
        Monster() {
            std::cout << "fuck yea";
        }
}

In [ ]:
send sync, receiv sync
send sync, receiv async
send async, receive sync
send async, receiv async

In [ ]:
class Model(nn.Module):
    def forward(self, x, labels):
        outputs = self.net(x)
        losses = loss_func(outputs, labels)
        return losses, outputs

In [ ]:
for _, data in enumerate(data_loader):
    outputs, losses = model(x)
    loss = loss.mean()
    loss.backward()
    optimizer.step()

In [ ]:
model parameters, gradients, optimiz

In [ ]:
step 1: model partritioning
step 2: move to rank
step 2.1: move to device
step 3: init local optimizer
step 4: step
step 5: broadcast
step 7: sync

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        self.num_embeddings = num_embeddings
        self.embedding_dim_per_partrition = embedding_dim // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.num_embeddings,
            self.embedding_dim_per_partrition
        ))
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            self.embedding_dim_per_partrition
        )
    
    def get_vocab_range(self, embedding_dim_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*embedding_dim_per_partrition
        end_idx = start_idx + embedding_dim_per_partrition
        return start_idx, end_idx
    
    def forward(self, tokens):
        mask = (self.vocab_start_idx > tokens) || (self.vocab_end_idx < tokens)
        tokens = tokens - self.vocab_start_idx
        tokens[mask] = 0.
        
        embeddings = F.embedding(tokens, self.weight)
        mask_idxs = torch.where(mask==False)[1]
        embeddings[mask_idxs] = 0.
        
        torch.distributed.all_reduce(embeddings)
        
        return embeddings

In [ ]:
step 1: partrition the parameters
step 2: to rank
step 3: init local optimizer
step 4: sync the optimizer's states
step 5:

In [ ]:
clock cycle 1: backward(m, n)
clock cycle 2: backward(m, n-1), backward(m-1, n)
clock cycle 3: backward(m, n-2), backward(m-1, n-1), backward(m-2, n)

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [ ]:
start.record()

hardshit()

end.record()

In [ ]:
start.time_elapsed(end)

In [7]:
class StartDependency(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        phony = torch.empty(1, requires_grad=False, device=input.device)
        return input, phony
    
    @staticmethod
    def backward(ctx, grad_input, grad_phony):
        return grad_input

In [8]:
class EndDependency(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, phony):
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return grad_input, None

In [9]:
def create_dependency(start_batch, end_batch):
    start_batch, phony = StartDependency.apply(start_batch)
    end_batch = EndDependency.apply(start_batch, phony)
    return start_batch, end_batch

In [ ]:
step 1: partrition
step 2: rank
step 3: init local
step 4: sync optimizer's states
step 5: move local params to buckets

In [11]:
class StartDependency(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        phony = torch.empty(
            0,
            requires_grad=False,
            device=input.device
        )
        
        return input, phony
    
    @staticmethod
    def backward(ctx, grad_input, grad_putput):
        return grad_input

In [ ]:
class EndDependency(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, phony):
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return input, None

In [10]:
def create_dependency(start_batch, end_batch):
    start_batch, phony = StartDependency.apply(start_batch)
    end_batch = EndDependency.apply(start_batch, phony)
    return start_batch, end_batch

In [ ]:
clock cycle 1: backward(m, n)
clock cycle 2: backward(m, n-1), backward(m-1, n)
clock cycle 3: backward(m, n-2), backward(m-1, n-1), backward(m-2, n)

In [ ]:
n_features = interference.shape[-1]

In [ ]:
interference[torch.arange(n_features), torch.arange(n_features)] = 0.

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, n, d_model):
        super().__init__()
        self.n, self.d_model = n, d_model
    
    def forward(self, tokens):
        n_tokens = len(tokens)
        embeddings = torch.zeros((n_tokens, self.d_model))
        
        for p in range(n_words):
            for i in range(self.d_model):
                denominator = torch.pow(self.n, (2*i/self.d_model))
                embeddings[p][i] = torch.sin(p/denominator) if i%2==0 else torch.cos(p/denominator)
        
        return embeddings

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [
            nn.Linear(n_in, nh),
            nn.ReLU(),
            nn.Linear(nh, n_out)
        ]
        
        for l in self.layers:
            self.add_module(l)

In [ ]:
from 

In [ ]:
n_features = interference.shape[-1]

In [ ]:
interference[torch.arange(n_features), torch.arange(n_features)] = 0.

In [15]:
from einops import einsum

In [ ]:
polysemanticity = inteference.pow(2).sum(dim=-1).sqrt()

In [16]:
from einops import repeat

In [ ]:
output = repeat(x, "h w -> h w n", n=2)

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, attention, d_model, n_heads):
        super().__init__()
        self.attention = attention
        self.d_model, self.n_heads = d_model, n_heads
        self.d_head = d_model // n_heads
    
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        self.proj = nn.Linear(d_model, d_model)
    
    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        return x.view(batch_size, self.n_heads, seq_len, self.d_head)
    
    def concat_heads(self, x):
        batch_size, n_heads, seq_len, d_head = x.size()
        return x.view(batch_size, seq_len, self.d_model)
    
    def forward(self, pre_q, prev_k, prev_v):
        q = self.w_q(pre_q)
        k = self.w_k(pre_k)
        v = self.w_v(pre_v)
        
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        attn_output, attn_weights = self.mha(q, k, v)
        
        output = self.concat_heads(attn_output)
        output = self.proj(output)
        
        return output, attn_weights

In [ ]:
def add_hook(model):
    for layer in model:
        layer.register_forward_hook(add_log)

In [18]:
def create_one_hot(idx, dim):
    x = torch.zeros(dim)
    x[idx] = 1.
    return x

In [19]:
create_one_hot(1, 3)

tensor([0., 1., 0.])

In [20]:
class PositionalEncoding(nn.Module):
    def __init__(self, n, d_model):
        super().__init__()
        self.n, self.d_model = n, d_model
    
    def forward(self, tokens):
        seq_len = len(tokens)
        embeddings = torch.zeros(seq_len, d_model)
        
        for p in range(seq_len):
            for i in range(self.d_model):
                denominator = torch.pow(self.n, 2*i/self.d_model)
                embeddings[p][i] = torch.sin(p/denominator) if i%2==0 else torch.cos(p/denominator)
        
        return embeddings

In [ ]:
n_features = interference.shape[-1]

In [ ]:
interference[torch.arange(n_features), torch.arange(n_features)] = 0.

In [ ]:
polysemanticity = interference.pow(2).sqrt().sum(dim=-1)

In [ ]:
step 1: two prompts
step 2: choose a component
step 3: record all inter activations
step 4: extract the activations of the prompt 1, patch
step 5: logit diff

In [ ]:
reward = get_reward(states, actions)

In [ ]:
def clip(ratio, epsilon):
    return torch.clamp(ratio, min=1-epsilon, max=1+epsilon)

In [ ]:
def calculate_loss_of_soft_v(states, actions, action_probs):
    q_preds = q_function(states, actions)
    entropy = -action_probs.log()
    
    v_values = v_function(states)
    
    return (v_values - (q_preds + entropy).mean()).pow(2).mean()

In [ ]:
from flax

In [22]:
from ray import tune
from ray.tune.callback import Callback

In [23]:
class PrintCallback(Callback):
    def on_trial_result(self, iteration, trials, trial, result, **info):
        print("..")

In [ ]:
tune.run(
    objective,
    config=search_space,
    callbacks=[PrintCallback()]
)